In [1]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
import tensorflow
# tensorflow.keras.__version__

C:\Users\tawnyn\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tawnyn\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tawnyn\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tawnyn\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import datasets

In [3]:
# # Read in csv (already transformed)
# df = pd.read_csv('..//..//data/hospitals/Display.csv')
# df.replace(True, 1, inplace=True)
# df

In [4]:
# Read in csv (already transformed)
hosp_df = pd.read_csv('..//..//data/hospitals/hosp19_util_data_prelim.csv')

# Drop columns not need for modeling
df = hosp_df.drop(['Description', 'FAC_NO', 'FAC_NAME','FAC_STR_ADDR','FAC_CITY','FAC_PHONE','FAC_ADMIN_NAME','FAC_OPERATED_THIS_YR','FAC_OP_PER_BEGIN_DT','FAC_OP_PER_END_DT','FAC_PAR_CORP_NAME','FAC_PAR_CORP_BUS_ADDR','FAC_PAR_CORP_CITY','FAC_PAR_CORP_STATE','REPT_PREP_NAME','SUBMITTED_DT','REV_REPT_PREP_NAME','REVISED_DT','CORRECTED_DT','LICENSE_NO','LICENSE_EFF_DATE','LICENSE_EXP_DATE','LICENSE_STATUS','FACILITY_LEVEL','ASSEMBLY_DIST','SENATE_DIST','CONGRESS_DIST','CENS_TRACT','MED_SVC_STUDY_AREA','LA_COUNTY_SVC_PLAN_AREA','COUNTY'], axis=1)
# ,'RURAL_HEALTH_CLINIC']

# Format Categorical Data
df.TEACH_HOSP = df.TEACH_HOSP != "No"
df.TEACH_RURAL = df.TEACH_RURAL != "No"
df.TRAUMA_CTR = df.TRAUMA_CTR != 0
df.HEALTH_SVC_AREA = df.HEALTH_SVC_AREA.str[:2]
df.LIC_CAT = df.LIC_CAT == "General Acute Care Hospital"
df.LICEE_TOC = df.LICEE_TOC == "Non-Profit Corporation (including church-related)"
df.PRIN_SERVICE_TYPE = df.PRIN_SERVICE_TYPE == "General Medical / Surgical"
df.FAC_ACQUIRE_EQUIP_OVER_500K = df.FAC_ACQUIRE_EQUIP_OVER_500K != "No"
df.OFFER_ALTERNATE_BIRTH_PROG = df.OFFER_ALTERNATE_BIRTH_PROG != "No"
df.LIC_CARDIOLOGY_CARDIOVASCULAR_SURG_SERVICES = df.LIC_CARDIOLOGY_CARDIOVASCULAR_SURG_SERVICES != 0
df.OFFER_AMBULATORY_SURG_PROG = df.OFFER_AMBULATORY_SURG_PROG != "No"
df.LIC_ED_LEV_END = df.LIC_ED_LEV_END != 0
df.EMSA_TRAUMA_DESIGNATION_PEDIATRIC = df.EMSA_TRAUMA_DESIGNATION_PEDIATRIC != 0
df.EMER_DEPT_AMBULANCE_DIVERSION_HOURS = df.EMER_DEPT_AMBULANCE_DIVERSION_HOURS != "No"
df.LIC_ED_LEV_BEGIN = df.LIC_ED_LEV_BEGIN != 0
df.EMSA_TRAUMA_DESIGNATION = df.EMSA_TRAUMA_DESIGNATION != 0
df.OUTPATIENT_PALLIATIVE_CARE_SERV_OFFERED = df.OUTPATIENT_PALLIATIVE_CARE_SERV_OFFERED != 0
df.INPATIENT_PALLIATIVE_CARE_PROG_OFFERED = df.INPATIENT_PALLIATIVE_CARE_PROG_OFFERED != 0
df.INPATIENT_HOSPICE_PROG_OFFERED = df.INPATIENT_HOSPICE_PROG_OFFERED != 0
df.SHORT_DOYLE_SERVICES_OFFERED = df.SHORT_DOYLE_SERVICES_OFFERED != 0


# df.PRIMARY_NON_ENGLISH_LANG = df.PRIMARY_NON_ENGLISH_LANG == "Spanish"
df.replace(True, 1, inplace=True)

# Drop final coloumns
df.drop(df.iloc[:, 267:323], inplace=True, axis=1)

# Fill in all blanks with 0 (for deep learning)
df = df.fillna(0)

df.head()

,FAC_ZIP,FAC_PAR_CORP_ZIP,TRAUMA_CTR,TEACH_HOSP,TEACH_RURAL,LONGITUDE,LATITUDE,HEALTH_SVC_AREA,LIC_CAT,LICEE_TOC,...,CARD_CATH_PED_IP_THER_VST,CARD_CATH_PED_OP_THER_VST,CARDIAC_CATHETERIZATION_ADULT_INPAT_THERAPEUTIC_VISITS,CARDIAC_CATHETERIZATION_ADULT_OUTPAT_THERAPEUTIC_VISITS,CARDIAC_CATHETERIZATION_THERAPEUTIC_VISITS_TOT,INPATIENT_AVG_PER_SURGERY,OUTPATIENT_AVG_PER_SURGERY,FAC_ACQUIRE_EQUIP_OVER_500K,Target,Label
0,94501,94602,1.0,0.0,0.0,-122.253991,37.76266,05,1.0,0.0,...,0,0,0,0,0,105.0,75.2,1.0,0.403166,1.0
1,94705,95833,1.0,0.0,0.0,-122.257430,37.85645,05,1.0,1.0,...,0,0,0,0,0,127.6,75.8,1.0,0.226161,0.0
2,94609,0,1.0,0.0,0.0,-122.267470,37.83722,05,1.0,1.0,...,22,35,2,1,60,137.4,70.8,0.0,0.608373,1.0
3,94578,94602,1.0,0.0,0.0,-122.118190,37.70648,05,1.0,0.0,...,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.0
4,94704,95833,1.0,0.0,0.0,-122.269840,37.86373,05,1.0,1.0,...,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.0


In [5]:
# Retrive all possible inputs
features = df.drop(["EMER_DEPT_VISITS_NOT_RESULT_ADMISSIONS_TOT", "ER_TRAFFIC_TOT","Target","Label"], axis=1) #df.iloc[:, 0:267]
features



,FAC_ZIP,FAC_PAR_CORP_ZIP,TRAUMA_CTR,TEACH_HOSP,TEACH_RURAL,LONGITUDE,LATITUDE,HEALTH_SVC_AREA,LIC_CAT,LICEE_TOC,...,CARDIOVASCULAR_SURG_OPER_ADULT_BYPASS_NOT_USED,CARDIOVASCULAR_SURG_OPER_BYPASS_NOT_USED_TOT,CARD_CATH_PED_IP_THER_VST,CARD_CATH_PED_OP_THER_VST,CARDIAC_CATHETERIZATION_ADULT_INPAT_THERAPEUTIC_VISITS,CARDIAC_CATHETERIZATION_ADULT_OUTPAT_THERAPEUTIC_VISITS,CARDIAC_CATHETERIZATION_THERAPEUTIC_VISITS_TOT,INPATIENT_AVG_PER_SURGERY,OUTPATIENT_AVG_PER_SURGERY,FAC_ACQUIRE_EQUIP_OVER_500K
0,94501.0,94602,1.0,0.0,0.0,-122.253991,37.762660,05,1.0,0.0,...,0,0,0,0,0,0,0,105.0,75.2,1.0
1,94705.0,95833,1.0,0.0,0.0,-122.257430,37.856450,05,1.0,1.0,...,0,0,0,0,0,0,0,127.6,75.8,1.0
2,94609.0,0,1.0,0.0,0.0,-122.267470,37.837220,05,1.0,1.0,...,3,34,22,35,2,1,60,137.4,70.8,0.0
3,94578.0,94602,1.0,0.0,0.0,-122.118190,37.706480,05,1.0,0.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,94704.0,95833,1.0,0.0,0.0,-122.269840,37.863730,05,1.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,95901.0,95991,1.0,0.0,0.0,-121.593602,39.138805,02,1.0,1.0,...,89,89,0,0,369,358,727,145.5,84.8,0.0
474,93703.0,93611,1.0,0.0,0.0,-119.779521,36.778140,09,1.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
475,93940.0,93940,1.0,0.0,0.0,-121.892428,36.580786,08,1.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
476,95023.0,95023,1.0,0.0,0.0,-121.386700,36.835140,08,1.0,0.0,...,0,0,0,0,0,0,0,0.0,0.0,0.0


## Data Pre-Processing

In [6]:
# Define X and y
X = features
y = df["Label"]
print(X.shape, y.shape)

(478, 265) (478,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=101)

In [8]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# # Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# # Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=265))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
tensorflow.keras.metrics.CategoricalAccuracy(
    name='categorical_accuracy', dtype=None
)

In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
#               metrics=[tf.keras.metrics.CategoricalAccuracy()])
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               26600     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 47,002
Trainable params: 47,002
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    validation_data=(X_test_scaled, y_test_categorical),
    epochs=500,
    shuffle=True,
    verbose=2
)

Train on 320 samples, validate on 158 samples
Epoch 1/500
320/320 - 0s - loss: 0.6785 - acc: 0.5656 - val_loss: 0.6011 - val_acc: 0.6646
Epoch 2/500
320/320 - 0s - loss: 0.5739 - acc: 0.7094 - val_loss: 0.4975 - val_acc: 0.8228
Epoch 3/500
320/320 - 0s - loss: 0.4464 - acc: 0.8438 - val_loss: 0.3883 - val_acc: 0.8544
Epoch 4/500
320/320 - 0s - loss: 0.3428 - acc: 0.8719 - val_loss: 0.3396 - val_acc: 0.8861
Epoch 5/500
320/320 - 0s - loss: 0.2836 - acc: 0.8750 - val_loss: 0.3347 - val_acc: 0.8608
Epoch 6/500
320/320 - 0s - loss: 0.2326 - acc: 0.9031 - val_loss: 0.3309 - val_acc: 0.8481
Epoch 7/500
320/320 - 0s - loss: 0.2138 - acc: 0.9125 - val_loss: 0.3580 - val_acc: 0.8481
Epoch 8/500
320/320 - 0s - loss: 0.1975 - acc: 0.9219 - val_loss: 0.4389 - val_acc: 0.7911
Epoch 9/500
320/320 - 0s - loss: 0.1584 - acc: 0.9375 - val_loss: 0.3756 - val_acc: 0.8354
Epoch 10/500
320/320 - 0s - loss: 0.1339 - acc: 0.9656 - val_loss: 0.3646 - val_acc: 0.8544
Epoch 11/500
320/320 - 0s - loss: 0.1074 - 

Epoch 88/500
320/320 - 0s - loss: 5.0468e-05 - acc: 1.0000 - val_loss: 1.2662 - val_acc: 0.8481
Epoch 89/500
320/320 - 0s - loss: 4.8947e-05 - acc: 1.0000 - val_loss: 1.2717 - val_acc: 0.8481
Epoch 90/500
320/320 - 0s - loss: 4.7611e-05 - acc: 1.0000 - val_loss: 1.2765 - val_acc: 0.8481
Epoch 91/500
320/320 - 0s - loss: 4.6836e-05 - acc: 1.0000 - val_loss: 1.2756 - val_acc: 0.8481
Epoch 92/500
320/320 - 0s - loss: 4.4781e-05 - acc: 1.0000 - val_loss: 1.2835 - val_acc: 0.8481
Epoch 93/500
320/320 - 0s - loss: 4.3795e-05 - acc: 1.0000 - val_loss: 1.2920 - val_acc: 0.8481
Epoch 94/500
320/320 - 0s - loss: 4.2317e-05 - acc: 1.0000 - val_loss: 1.2931 - val_acc: 0.8481
Epoch 95/500
320/320 - 0s - loss: 4.1101e-05 - acc: 1.0000 - val_loss: 1.2964 - val_acc: 0.8481
Epoch 96/500
320/320 - 0s - loss: 4.0034e-05 - acc: 1.0000 - val_loss: 1.3005 - val_acc: 0.8481
Epoch 97/500
320/320 - 0s - loss: 3.8791e-05 - acc: 1.0000 - val_loss: 1.3065 - val_acc: 0.8481
Epoch 98/500
320/320 - 0s - loss: 3.7872

Epoch 173/500
320/320 - 0s - loss: 8.9142e-06 - acc: 1.0000 - val_loss: 1.5406 - val_acc: 0.8481
Epoch 174/500
320/320 - 0s - loss: 8.7857e-06 - acc: 1.0000 - val_loss: 1.5435 - val_acc: 0.8481
Epoch 175/500
320/320 - 0s - loss: 8.6687e-06 - acc: 1.0000 - val_loss: 1.5456 - val_acc: 0.8481
Epoch 176/500
320/320 - 0s - loss: 8.5365e-06 - acc: 1.0000 - val_loss: 1.5480 - val_acc: 0.8481
Epoch 177/500
320/320 - 0s - loss: 8.4560e-06 - acc: 1.0000 - val_loss: 1.5513 - val_acc: 0.8481
Epoch 178/500
320/320 - 0s - loss: 8.3152e-06 - acc: 1.0000 - val_loss: 1.5513 - val_acc: 0.8481
Epoch 179/500
320/320 - 0s - loss: 8.1968e-06 - acc: 1.0000 - val_loss: 1.5528 - val_acc: 0.8481
Epoch 180/500
320/320 - 0s - loss: 8.1055e-06 - acc: 1.0000 - val_loss: 1.5576 - val_acc: 0.8481
Epoch 181/500
320/320 - 0s - loss: 7.9677e-06 - acc: 1.0000 - val_loss: 1.5597 - val_acc: 0.8481
Epoch 182/500
320/320 - 0s - loss: 7.8604e-06 - acc: 1.0000 - val_loss: 1.5615 - val_acc: 0.8481
Epoch 183/500
320/320 - 0s - l

Epoch 258/500
320/320 - 0s - loss: 3.2625e-06 - acc: 1.0000 - val_loss: 1.7049 - val_acc: 0.8481
Epoch 259/500
320/320 - 0s - loss: 3.2305e-06 - acc: 1.0000 - val_loss: 1.7062 - val_acc: 0.8481
Epoch 260/500
320/320 - 0s - loss: 3.1992e-06 - acc: 1.0000 - val_loss: 1.7086 - val_acc: 0.8481
Epoch 261/500
320/320 - 0s - loss: 3.1664e-06 - acc: 1.0000 - val_loss: 1.7096 - val_acc: 0.8481
Epoch 262/500
320/320 - 0s - loss: 3.1344e-06 - acc: 1.0000 - val_loss: 1.7114 - val_acc: 0.8481
Epoch 263/500
320/320 - 0s - loss: 3.1027e-06 - acc: 1.0000 - val_loss: 1.7137 - val_acc: 0.8481
Epoch 264/500
320/320 - 0s - loss: 3.0714e-06 - acc: 1.0000 - val_loss: 1.7150 - val_acc: 0.8481
Epoch 265/500
320/320 - 0s - loss: 3.0413e-06 - acc: 1.0000 - val_loss: 1.7165 - val_acc: 0.8481
Epoch 266/500
320/320 - 0s - loss: 3.0133e-06 - acc: 1.0000 - val_loss: 1.7180 - val_acc: 0.8481
Epoch 267/500
320/320 - 0s - loss: 2.9839e-06 - acc: 1.0000 - val_loss: 1.7197 - val_acc: 0.8481
Epoch 268/500
320/320 - 0s - l

Epoch 343/500
320/320 - 0s - loss: 1.5125e-06 - acc: 1.0000 - val_loss: 1.8347 - val_acc: 0.8481
Epoch 344/500
320/320 - 0s - loss: 1.4998e-06 - acc: 1.0000 - val_loss: 1.8368 - val_acc: 0.8418
Epoch 345/500
320/320 - 0s - loss: 1.4871e-06 - acc: 1.0000 - val_loss: 1.8378 - val_acc: 0.8481
Epoch 346/500
320/320 - 0s - loss: 1.4767e-06 - acc: 1.0000 - val_loss: 1.8396 - val_acc: 0.8418
Epoch 347/500
320/320 - 0s - loss: 1.4622e-06 - acc: 1.0000 - val_loss: 1.8407 - val_acc: 0.8481
Epoch 348/500
320/320 - 0s - loss: 1.4502e-06 - acc: 1.0000 - val_loss: 1.8425 - val_acc: 0.8418
Epoch 349/500
320/320 - 0s - loss: 1.4406e-06 - acc: 1.0000 - val_loss: 1.8431 - val_acc: 0.8481
Epoch 350/500
320/320 - 0s - loss: 1.4294e-06 - acc: 1.0000 - val_loss: 1.8444 - val_acc: 0.8481
Epoch 351/500
320/320 - 0s - loss: 1.4149e-06 - acc: 1.0000 - val_loss: 1.8458 - val_acc: 0.8481
Epoch 352/500
320/320 - 0s - loss: 1.4059e-06 - acc: 1.0000 - val_loss: 1.8477 - val_acc: 0.8418
Epoch 353/500
320/320 - 0s - l

Epoch 428/500
320/320 - 0s - loss: 7.8491e-07 - acc: 1.0000 - val_loss: 1.9465 - val_acc: 0.8481
Epoch 429/500
320/320 - 0s - loss: 7.8044e-07 - acc: 1.0000 - val_loss: 1.9473 - val_acc: 0.8481
Epoch 430/500
320/320 - 0s - loss: 7.7262e-07 - acc: 1.0000 - val_loss: 1.9491 - val_acc: 0.8481
Epoch 431/500
320/320 - 0s - loss: 7.6703e-07 - acc: 1.0000 - val_loss: 1.9504 - val_acc: 0.8481
Epoch 432/500
320/320 - 0s - loss: 7.6405e-07 - acc: 1.0000 - val_loss: 1.9517 - val_acc: 0.8481
Epoch 433/500
320/320 - 0s - loss: 7.5772e-07 - acc: 1.0000 - val_loss: 1.9526 - val_acc: 0.8481
Epoch 434/500
320/320 - 0s - loss: 7.5325e-07 - acc: 1.0000 - val_loss: 1.9536 - val_acc: 0.8481
Epoch 435/500
320/320 - 0s - loss: 7.4803e-07 - acc: 1.0000 - val_loss: 1.9551 - val_acc: 0.8481
Epoch 436/500
320/320 - 0s - loss: 7.4096e-07 - acc: 1.0000 - val_loss: 1.9568 - val_acc: 0.8481
Epoch 437/500
320/320 - 0s - loss: 7.3649e-07 - acc: 1.0000 - val_loss: 1.9576 - val_acc: 0.8481
Epoch 438/500
320/320 - 0s - l

## Quantify our Trained Model

In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

158/158 - 0s - loss: 2.0328 - acc: 0.8481
Normal Neural Network - Loss: 2.032788979856274, Accuracy: 0.8481012582778931


## Make Predictions

In [38]:
encoded_predictions = model.predict_classes(X_test_scaled[:2])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [39]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:2])}")

Predicted classes: [0. 0.]
Actual Labels: [0.0, 0.0]


In [20]:
from sklearn.metrics import confusion_matrix
y_true = y_test[:5]
y_pred = prediction_labels
confusion_matrix(y_true, y_pred)


array([[3, 0],
       [1, 1]], dtype=int64)

In [21]:
from sklearn.metrics import cohen_kappa_score
y_true = y_test[:5]
y_pred = prediction_labels
cohen_kappa_score(y_true, y_pred)

0.5454545454545454

In [22]:
predictions = model.predict_classes(X_test_scaled)
labels = label_encoder.inverse_transform(predictions)
actual = list(labels)
predicted = list(y_test)

In [23]:
from sklearn.metrics import confusion_matrix
y_true = actual
y_pred = predicted
confusion_matrix(y_true, y_pred)

array([[86,  7],
       [17, 48]], dtype=int64)

In [24]:
from sklearn.metrics import classification_report
y_true = actual
y_pred = predicted
target_names = ['class 0 - false', 'class 1 - true']
print(classification_report(y_true, y_pred, target_names=target_names))

                 precision    recall  f1-score   support

class 0 - false       0.83      0.92      0.88        93
 class 1 - true       0.87      0.74      0.80        65

       accuracy                           0.85       158
      macro avg       0.85      0.83      0.84       158
   weighted avg       0.85      0.85      0.85       158



In [ ]:
# Create the SVC Model
from sklearn.svm import SVC 
model2 = SVC(kernel='linear')
model2.fit(X_train_scaled, y_train)

# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model2, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import make_scorer
# scoring = {'accuracy': make_scorer(accuracy_score),
#             'prec': 'precision'}
# scoring

In [ ]:
results = grid.cv_results_
results

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                  param_grid={'min_samples_split': range(2, 403, 10)},
                  scoring=scoring, refit='AUC', return_train_score=True)
gs.fit(X, y)

In [ ]:
results = gs.cv_results_
results

In [ ]:
print(gs.best_params_)
print(gs.best_score_)

In [ ]:
gs.predict_proba(X)

In [ ]:
gs.predict_log_proba(X)

In [ ]:
gs.get_params(deep=True)